## 2-1 read_html 연습.html 실습

In [1]:
import pandas as pd

In [2]:
table = pd.read_html('https://mizykk.tistory.com/39', header=0, encoding='utf-8')

type(table)

list

In [3]:
table

[    과일  색상     가격
 0   사과  빨강  1500원
 1   사과  초록  1000원
 2  바나나  노랑  3000원
 3  바나나  초록  2000원,
      국가   도시     언어
 0    한국   서울    한국어
 1    미국   뉴욕     영어
 2  이탈리아  피렌체  이탈리아어
 3   프랑스   파리     불어,
   한국어        영어
 0  1월   January
 1  2월  Feburary
 2  3월     March,
               [Python] 리스트 정렬하기 :: sort / sorted (0)  2021.02.10
 0    [Python] 영문 > 한글 키보드 입력 변환하기 :: 자모결합 / 자소결합 (0)  2021.01.29
 1  [Python] for문(반복문) 진행상황 표시 :: tqdm / tqdm_note...  2021.01.28
 2  [Python] has no attribute 'compat' (most likel...  2020.10.05
 3                               pd.read_table 예제 (0)  2020.03.31
 4               [Python] 위경도 거리 구하기 :: haversine (1)  2020.01.28]

In [4]:
len(table)

4

In [5]:
table0 = pd.read_html('https://mizykk.tistory.com/39', header=0, encoding='utf-8')[0]
table0

,과일,색상,가격
0,사과,빨강,1500원
1,사과,초록,1000원
2,바나나,노랑,3000원
3,바나나,초록,2000원


In [6]:
table1 = pd.read_html('https://mizykk.tistory.com/39', header=0, encoding='utf-8')[1]
table1

,국가,도시,언어
0,한국,서울,한국어
1,미국,뉴욕,영어
2,이탈리아,피렌체,이탈리아어
3,프랑스,파리,불어


In [7]:
table2 = pd.read_html('https://mizykk.tistory.com/39', header=0, encoding='utf-8')[2]
table2

,한국어,영어
0,1월,January
1,2월,Feburary
2,3월,March


In [8]:
table3 = pd.read_html('https://mizykk.tistory.com/39', header=0, encoding='utf-8')[3]
table3

,[Python] 리스트 정렬하기 :: sort / sorted (0),2021.02.10
0,[Python] 영문 > 한글 키보드 입력 변환하기 :: 자모결합 / 자소결합 (0),2021.01.29
1,[Python] for문(반복문) 진행상황 표시 :: tqdm / tqdm_note...,2021.01.28
2,[Python] has no attribute 'compat' (most likel...,2020.10.05
3,pd.read_table 예제 (0),2020.03.31
4,[Python] 위경도 거리 구하기 :: haversine (1),2020.01.28


In [9]:
table4 = pd.read_html('https://mizykk.tistory.com/39', header=0, encoding='utf-8')[4]
table4

IndexError: list index out of range

## 2-2 read_html 로 종목 읽어오기.html 실습

In [10]:
Mypassword = 'abcd123456'

In [11]:
def read_krx_code():
    """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
        'download&searchType=13'
    krx = pd.read_html(url, header=0)[0]
    krx = krx[['종목코드', '회사명']]
    krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
    krx.code = krx.code.map('{:06d}'.format)
    return krx

In [12]:
read_krx_code()

,code,company
0,000210,DL
1,004840,DRB동일
2,155660,DSR
3,078930,GS
4,001390,KG케미칼
...,...,...
2496,393210,토마토시스템
2497,217880,틸론
2498,222670,플럼라인생명과학
2499,331660,한국미라클피플사


In [13]:
import pymysql
from datetime import datetime

In [14]:
conn = pymysql.connect(host='localhost', user='root', 
                       password=Mypassword, db='whsf1_db', charset='utf8')

"""종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
sql = "SELECT * FROM company_info"

#codes = dict()
curs = conn.cursor()

df = pd.read_sql(sql, conn)

In [15]:
df

,code,company,last_update
0,000020,동화약품,2022-04-12
1,000040,KR모터스,2022-04-12
2,000050,경방,2022-04-12
3,000060,메리츠화재,2022-04-12
4,000070,삼양홀딩스,2022-04-12
...,...,...,...
2496,950180,SNK,2022-04-12
2497,950190,미투젠,2022-04-12
2498,950200,소마젠,2022-04-12
2499,950210,프레스티지바이오파마,2022-04-12


In [16]:
sql = "SELECT max(last_update) FROM company_info"
curs.execute(sql)

# DB에서 가져온 첫번째 결과
rs = curs.fetchone()
print(rs)

# 오늘 날짜 및 현재시간
var = datetime.today()
print(var)

# 오늘 날짜 부분만 today에 할당
today = datetime.today().strftime('%Y-%m-%d')
print(today)

(datetime.date(2022, 4, 12),)
2022-04-13 00:30:57.836668
2022-04-13


In [17]:
if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:   # 입력된 값이 없거나 입력된 날짜가 오늘 이전인 경우
    krx = read_krx_code()                                  # 오늘의 종목을 읽어옵니다.

    for idx in range(len(krx)):                           # 종목수만큼
        code = krx.code.values[idx]                       # 코드를 읽어옴
        print(code)
        company = krx.company.values[idx]                 # 기업명을 읽어옴
        print(company)
        
        # python의 format 문. code, company, last_update 값 지정
        
        sql = f"REPLACE INTO company_info (code, company, last"\
            f"_update) VALUES ('{code}', '{company}', '{today}')"
        curs.execute(sql)

        tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
        
        # 현재 시각 00번 째 정보 입력된다는 메세지 출력
        print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
            f"VALUES ({code}, {company}, {today})")
    conn.commit()


conn.close()

000210
DL
[2022-04-13 00:30] #0001 REPLACE INTO company_info VALUES (000210, DL, 2022-04-13)
004840
DRB동일
[2022-04-13 00:30] #0002 REPLACE INTO company_info VALUES (004840, DRB동일, 2022-04-13)
155660
DSR
[2022-04-13 00:30] #0003 REPLACE INTO company_info VALUES (155660, DSR, 2022-04-13)
078930
GS
[2022-04-13 00:30] #0004 REPLACE INTO company_info VALUES (078930, GS, 2022-04-13)
001390
KG케미칼
[2022-04-13 00:30] #0005 REPLACE INTO company_info VALUES (001390, KG케미칼, 2022-04-13)
025000
KPX케미칼
[2022-04-13 00:30] #0006 REPLACE INTO company_info VALUES (025000, KPX케미칼, 2022-04-13)
058860
KTis
[2022-04-13 00:30] #0007 REPLACE INTO company_info VALUES (058860, KTis, 2022-04-13)
011070
LG이노텍
[2022-04-13 00:30] #0008 REPLACE INTO company_info VALUES (011070, LG이노텍, 2022-04-13)
037560
LG헬로비전
[2022-04-13 00:30] #0009 REPLACE INTO company_info VALUES (037560, LG헬로비전, 2022-04-13)
108670
LX하우시스
[2022-04-13 00:30] #0010 REPLACE INTO company_info VALUES (108670, LX하우시스, 2022-04-13)
383800
LX홀딩스
[2022-04-

In [18]:
conn = pymysql.connect(host='localhost', user='root', 
                       password=Mypassword, db='whsf1_db', charset='utf8')

"""종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
sql = "SELECT * FROM company_info"

#codes = dict()
curs = conn.cursor()

df = pd.read_sql(sql, conn)
conn.close()

In [19]:
df

,code,company,last_update
0,000020,동화약품,2022-04-13
1,000040,KR모터스,2022-04-13
2,000050,경방,2022-04-13
3,000060,메리츠화재,2022-04-13
4,000070,삼양홀딩스,2022-04-13
...,...,...,...
2496,950180,SNK,2022-04-13
2497,950190,미투젠,2022-04-13
2498,950200,소마젠,2022-04-13
2499,950210,프레스티지바이오파마,2022-04-13


## 2-3 Python으로 Database 다루기.html 실습

In [20]:
import pymysql

- DB와의 통로 열음

In [21]:
conn = pymysql.connect(host='localhost', port=3306, db='whsf1_db', user='root', passwd=Mypassword, autocommit=True)

- 실행결과를 cursor에 담음

In [22]:
cursor = conn.cursor()

- DB version 번호 검사

In [23]:
cursor.execute('select version();')

1

- DB에서 가져온 결과 중 한 줄 가져옴

In [24]:
rs = cursor.fetchone()

- Version 번호 인쇄

In [25]:
print("MariaDB version: {}".format(rs))

MariaDB version: ('10.6.7-MariaDB',)


- sql 문장 정의

In [26]:
sql1 = 'show tables'

- sql 문장 실행

In [27]:
cursor.execute(sql1)

2

In [28]:
rs = cursor.fetchone()
print('tables: ', rs)

tables:  ('company_info',)


- company_info 테이블 내의 데이터 읽어오기

In [29]:
sql2 = 'select * from company_info'
cursor.execute(sql2)

2501

In [30]:
rs = cursor.fetchone()
rs

('000020', '동화약품', datetime.date(2022, 4, 13))

- company_info 테이블 내용 모두 삭제

In [31]:
sql_delete_all ="delete from company_info"
cursor.execute(sql_delete_all)

2501

In [32]:
rs=cursor.fetchone()
rs

In [33]:
sql_drop_table ="drop table company_info"
cursor.execute(sql_drop_table)

0

In [34]:
rs = cursor.fetchone()
print(rs)

None


In [35]:
sql_create_table ="""
CREATE TABLE if not exists company_info (
    code VARCHAR(20),
    company VARCHAR(40),
    last_update DATE,
    PRIMARY KEY (code)
  );
 """
cursor.execute(sql_create_table)

0

In [36]:
sql1 = "show tables"
cursor.execute(sql1)

2

In [37]:
rs = cursor.fetchone()
print("tables: ", rs)

tables:  ('company_info',)


In [38]:
rs = cursor.fetchone()
print("tables: ", rs)

tables:  ('daily_price',)


In [39]:
rs = cursor.fetchone()
print("tables: ", rs)

tables:  None


In [40]:
conn.close()

##  2-4 종목 가져와서 저장.html 실습

In [41]:
def read_krx_code():
    """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
        'download&searchType=13'
    krx = pd.read_html(url, header=0)[0]
    krx = krx[['종목코드', '회사명']]
    krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
    krx.code = krx.code.map('{:06d}'.format)
    return krx

In [42]:
read_krx_code()

,code,company
0,000210,DL
1,004840,DRB동일
2,155660,DSR
3,078930,GS
4,001390,KG케미칼
...,...,...
2496,393210,토마토시스템
2497,217880,틸론
2498,222670,플럼라인생명과학
2499,331660,한국미라클피플사


In [43]:
import pymysql
from datetime import datetime

In [44]:
conn = pymysql.connect(host='localhost', user='root', 
                       password=Mypassword, db='whsf1_db', charset='utf8')

"""종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
sql = "SELECT * FROM company_info"

#codes = dict()
curs = conn.cursor()

df = pd.read_sql(sql, conn)

In [45]:
df

,code,company,last_update


In [46]:
sql = "SELECT max(last_update) FROM company_info"
curs.execute(sql)

# DB에서 가져온 첫번째 결과
rs = curs.fetchone()
print(rs)

# 오늘 날짜 및 현재시간
var = datetime.today()
print(var)

# 오늘 날짜 부분만 today에 할당
today = datetime.today().strftime('%Y-%m-%d')
print(today)

(None,)
2022-04-13 00:31:26.418209
2022-04-13


In [47]:
if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:   # 입력된 값이 없거나 입력된 날짜가 오늘 이전인 경우
    krx = read_krx_code()                                  # 오늘의 종목을 읽어옵니다.

    for idx in range(len(krx)):                           # 종목수만큼
        code = krx.code.values[idx]                       # 코드를 읽어옴
        print(code)
        company = krx.company.values[idx]                 # 기업명을 읽어옴
        print(company)
        
        # python의 format 문. code, company, last_update 값 지정
        
        sql = f"REPLACE INTO company_info (code, company, last"\
            f"_update) VALUES ('{code}', '{company}', '{today}')"
        curs.execute(sql)

        tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
        
        # 현재 시각 00번 째 정보 입력된다는 메세지 출력
        print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
            f"VALUES ({code}, {company}, {today})")
    conn.commit()


conn.close()

000210
DL
[2022-04-13 00:31] #0001 REPLACE INTO company_info VALUES (000210, DL, 2022-04-13)
004840
DRB동일
[2022-04-13 00:31] #0002 REPLACE INTO company_info VALUES (004840, DRB동일, 2022-04-13)
155660
DSR
[2022-04-13 00:31] #0003 REPLACE INTO company_info VALUES (155660, DSR, 2022-04-13)
078930
GS
[2022-04-13 00:31] #0004 REPLACE INTO company_info VALUES (078930, GS, 2022-04-13)
001390
KG케미칼
[2022-04-13 00:31] #0005 REPLACE INTO company_info VALUES (001390, KG케미칼, 2022-04-13)
025000
KPX케미칼
[2022-04-13 00:31] #0006 REPLACE INTO company_info VALUES (025000, KPX케미칼, 2022-04-13)
058860
KTis
[2022-04-13 00:31] #0007 REPLACE INTO company_info VALUES (058860, KTis, 2022-04-13)
011070
LG이노텍
[2022-04-13 00:31] #0008 REPLACE INTO company_info VALUES (011070, LG이노텍, 2022-04-13)
037560
LG헬로비전
[2022-04-13 00:31] #0009 REPLACE INTO company_info VALUES (037560, LG헬로비전, 2022-04-13)
108670
LX하우시스
[2022-04-13 00:31] #0010 REPLACE INTO company_info VALUES (108670, LX하우시스, 2022-04-13)
383800
LX홀딩스
[2022-04-

In [48]:
conn = pymysql.connect(host='localhost', user='root', 
                       password=Mypassword, db='whsf1_db', charset='utf8')

"""종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
sql = "SELECT * FROM company_info"

#codes = dict()
curs = conn.cursor()

df = pd.read_sql(sql, conn)
conn.close()

In [49]:
df

,code,company,last_update
0,000020,동화약품,2022-04-13
1,000040,KR모터스,2022-04-13
2,000050,경방,2022-04-13
3,000060,메리츠화재,2022-04-13
4,000070,삼양홀딩스,2022-04-13
...,...,...,...
2496,950180,SNK,2022-04-13
2497,950190,미투젠,2022-04-13
2498,950200,소마젠,2022-04-13
2499,950210,프레스티지바이오파마,2022-04-13
